# MiniPM: AI-Powered Project Management Assistant

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>

## Description:

MiniPM is an AI-powered tool designed to assist project managers in generating Product Requirements Documents (PRDs) and User Stories for their projects. It simplifies the documentation process by leveraging Groq LLM, helping teams structure their project details efficiently. By automating the creation of these essential documents, MiniPM enhances productivity, ensuring well-organized project planning and execution.




## Step 1: Install Requirements

This function handles the installation of dependencies from a `requirements.txt` file. It checks if the requirements have already been installed to avoid redundant installations. If not, it attempts to install them using the `pip install -r requirements.txt` command. In case of failure, it retries up to a specified maximum number of times (3 retries in this case) before exiting the program. The function ensures the environment is ready by installing necessary packages to run the application smoothly.

In [1]:
import os

requirements_installed = False
max_retries = 3
retries = 0


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return

The function install_requirements() is designed to install the required Python packages listed in the requirements.txt file. 

In [ ]:
install_requirements()

## Step 2: Environment Setup and API Key Validation

The `setup_env()` function loads environment variables from a `.env` file using `load_dotenv()` and retrieves the `GROQ_API_KEY` using `os.getenv()`. If the API key is not found, it prompts the user to set it and terminates the program. If the key is set, it prints a success message confirming the setup, ensuring the necessary credentials are available for the application to proceed securely.

In [3]:
from dotenv import load_dotenv
import os


def setup_env():
    """Sets up the environment variables"""
    load_dotenv()

    GROQ_API_KEY = os.getenv("GROQ_API_KEY")

    if GROQ_API_KEY is None:
        print("Please set the GROQ_API_KEY environment variable.")
        exit(1)
    else:
        print("GROQ_API_KEY is set.")

The setup_env() function loads environment variables from a .env file using load_dotenv() to ensure the necessary configurations are available. It then checks for the GROQ_API_KEY in the environment variables, prompting the user to set it if missing. If the key is present, a confirmation message is printed, ensuring the API can be accessed with valid credentials.

In [ ]:
setup_env()

## Step 3: LLM API Interaction and Response Handling

This code defines functions for interacting with the Groq LLM API to process user prompts. The `get_groq_client()` function initializes the Groq client with the API key and returns the client instance. The `llm()` function sends a user prompt to the API, retrieves a response, and returns it in the specified format. In case of an error during the process, it catches the exception and returns a structured error response.

In [5]:
import instructor
from groq import Groq
import traceback
from pydantic import BaseModel
from typing import Union

DEFAULT_MODEL = "llama-3.3-70b-versatile"


class LLMErrorResponse(BaseModel):
    error: str


def get_groq_client():
    """Returns an instance of the Groq class"""
    groq = Groq(api_key=os.getenv("GROQ_API_KEY"))
    client = instructor.from_groq(groq, mode=instructor.Mode.JSON)
    return client


def llm(
    prompt: str,
    response_model: BaseModel,
    system="You are a helpful AI assistant. The user will talk to you and its your job to provide detailed and clear responses.",
    model=DEFAULT_MODEL,
) -> Union[BaseModel, LLMErrorResponse]:
    """Calls LLM API with the given prompt. Defaults to llama-3.3-70b-versatile""",
    try:
        client = get_groq_client()

        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ]

        response = client.chat.completions.create(
            messages=messages, model=model, response_model=response_model
        )
        return response
    except Exception as e:
        traceback.print_exc()
        return LLMErrorResponse(error=str(e))

## Step 4: Product Requirements Document (PRD) and User Story Generation

The **MiniPM** class is an AI-powered tool designed to assist project managers in generating a **Product Requirements Document (PRD)** and **User Stories**. 

### Key Features:

- **`get_prd()` Method**:  

  Generates a **Product Requirements Document (PRD)** based on a project description. The PRD can be returned in various formats, such as:
  
  - JSON

  - Markdown

  - DTO

- **`_augment_prd()` Method**:  

  Enhances the generated PRD by adding more detailed information to each section, ensuring thoroughness and clarity.

- **`get_user_stories()` Method**:  

  Generates **User Stories** based on the project details and the generated PRD, helping to break down the project into actionable tasks.

### Core Technology:

- **Groq LLM Integration**:  

  All functionalities are powered by the Groq Large Language Model (LLM), which processes project details and generates the necessary documents and stories with high accuracy.

### Use Cases:

- **Product Managers**:  

  Helps create well-structured PRDs and User Stories quickly, enhancing productivity.
  
- **Project Teams**:  

  Breaks down complex projects into actionable user stories, ensuring clear objectives and goals.

### Summary:

The MiniPM class simplifies the process of creating essential project documents like PRDs and User Stories, offering a streamlined solution powered by AI, which is especially useful for product managers and teams looking to enhance their project planning and execution.


In [6]:
from typing import List, Union, Any
from pydantic import BaseModel
from groq import Groq


class UserStory(BaseModel):
    """Represents a User Story"""

    title: str
    description: str
    acceptance_criteria: List[str]
    story_points: int


class UserStories(BaseModel):
    """Represents a collection of User Stories"""

    user_stories: List[UserStory]


class PRD(BaseModel):
    """Represents a Product Requirements Document (PRD)"""

    title: str
    overview: str
    problem_statement: str
    business_domains: List[str]
    proposed_solution: str
    target_audience: str
    features: List[str]
    constraints: List[str]
    assumptions: List[str]
    dependencies: List[str]
    user_personas: List[str]
    conclusion: str


class MiniPM:
    """An AI agent that acts as a Product Manager to help project managers create PRD and User Stories for their projects."""

    llm: Groq
    project_description: str

    def __init__(self, project_description: str):
        """Initializes the MiniPM AI agent with the project description."""
        self.project_description = project_description
        self.llm = get_groq_client()

    def get_prd(
        self, response_format="markdown", augment=True
    ) -> Union[PRD, LLMErrorResponse, Any]:
        """Gets the Product Requirements Document (PRD) for the project."""
        system = """
                You are MiniPM; a Product Manager AI agent. 
                You help project managers to create a Product Requirements Document (PRD) for their projects.
                """
        prompt = f"""
                Provide a Product Requirements Document (PRD) for the project with the following details:
                Details: {self.project_description}
                """
        response = llm(prompt=prompt, response_model=PRD, system=system)

        if augment:
            response = self._augment_prd(response)

        if response_format == "json":
            return response.model_dump_json()
        elif response_format == "markdown":
            response_dict = response.model_dump()
            keys = response_dict.keys()
            markdown = f"# Product Requirements Document (PRD) for {response.title}\n"
            for key in keys:
                title = key.title().replace("_", " ").replace("Prd", "PRD")
                if key == "title":
                    title = ""
                    continue
                markdown += f"## {title}\n"
                value = response_dict[key]
                if type(value) == list:
                    value_bullets = [f"- {item}" for item in value]
                    value = "\n".join(value_bullets)
                    markdown += f"{value}\n"
                else:
                    markdown += f"{response_dict[key]}\n"
            return markdown
        elif response_format == "dto":
            return response
        raise ValueError("Invalid response_format. Use 'json', 'markdown' or 'dto'.")

    def _augment_prd(self, prd: PRD) -> PRD:
        """Augments the PRD with greater details."""
        prd_json = prd.model_dump()
        print("Augmenting PRD: ", prd_json)
        keys = prd_json.keys()
        for key in keys:
            if key == "title":
                continue
            value = prd_json[key]
            prompt = f"Provide more details about the {key} for the project."
            if type(value) == str:
                prompt += f" Current value: {value}"
                expanded_value = llm(prompt, response_model=str)
                prd_json[key] = expanded_value
            elif type(value) == list:
                expanded_values = []
                for item in value:
                    prompt += f" Current value: {item}"
                    expanded_item = llm(prompt, response_model=str)
                    expanded_values.append(expanded_item)
                prd_json[key] = expanded_values
        print("Augmented PRD: ", prd_json)
        return PRD(**prd_json)

    def get_user_stories(self) -> Union[UserStories, LLMErrorResponse]:
        """Gets the User Stories for the project"""
        system = """
                You are MiniPM; a Product Manager AI agent. 
                You help project managers to create User Stories for their projects.
                """
        prompt = f"""
                Provide User Stories for the project with the following details:
                Details: {self.project_description}
                PRD: {self.get_prd()}
                """
        response = llm(prompt=prompt, response_model=UserStories, system=system)
        return response

## Step 5: Render and Save Markdown Output

The `render_output()` function takes the generated markdown text as input and renders it for display using IPython's `Markdown` class, which allows it to be viewed in a Jupyter notebook interface. The `save_markdown_file()` function takes the markdown content and saves it to a specified file path, enabling the user to persist the generated content as a `.md` file for later use or distribution.

In [7]:
from IPython.display import Markdown


def render_output(markdown: str) -> None:
    """Renders the generated output file as markdown."""
    return Markdown(markdown)


def save_markdown_file(markdown: str, file_path: str) -> None:
    """Saves the generated markdown file to the specified path."""
    with open(file_path, "w") as file:
        file.write(markdown)

## Step 6: Generate and Save PRD for Project

In this step, the `project_description` is set to a specific project idea (a web application for tracking behavioral changes in aliens). A `MiniPM` instance is created with this project description, and the Product Requirements Document (PRD) is fetched using the `get_prd()` method. The PRD is then saved to a markdown file using `save_markdown_file()`. Finally, the generated PRD markdown content is rendered using `render_output()` for viewing in a Jupyter notebook or similar interface.

In [ ]:
## Input Parameters: Change these values to get different PRD and User Stories
project_description = (
    "A web application for tracking investigating behavioural change in aliens."
)
output_file = "alien_behavioural_change_prd.md"
augment = True

## Create an instance of MiniPM AI agent
minipm = MiniPM(project_description)

## Get the Product Requirements Document (PRD)
prd = minipm.get_prd(response_format="markdown", augment=augment)

save_markdown_file(prd, output_file)

## Render the PRD as markdown
render_output(prd)

## Step 7: Generate User Stories for Project

In this step, the `project_description` is set to a new idea (a web application for tracking fitness goals and workouts). An instance of the `MiniPM` AI agent is created with this project description, and the User Stories are fetched using the `get_user_stories()` method. Finally, the generated User Stories are printed in a JSON format using `model_dump_json()` for easy visualization.

In [ ]:
## Input Parameters: Change these values to get different PRD and User Stories
project_description = "A web application for tracking fitness goals and workouts."

## Create an instance of MiniPM AI agent
minipm = MiniPM(project_description)

## Get the Product Requirements Document (PRD)
user_stories = minipm.get_user_stories()

## Render the PRD as markdown
print(user_stories.model_dump_json(indent=4))

## Conclusion:

**MiniPM** is an AI-powered tool designed to automate and simplify the creation of crucial project documentation, including **Product Requirements Documents (PRDs)** and **User Stories**. By leveraging the **Groq LLM**, MiniPM provides a structured approach to project planning, helping teams save time and focus on execution.

### Key Features:

- **Automated PRD Generation**:  
  Quickly generates comprehensive **PRDs**, ensuring that all project details, such as business goals, target audience, and features, are clearly defined.

- **User Story Creation**:  
  Breaks down complex projects into **actionable User Stories**, making task assignment, prioritization, and tracking more efficient.

- **Enhanced Collaboration**:  
  Fosters clear communication between team members, stakeholders, and business leaders by producing structured and detailed documentation.

- **Time Efficiency**:  
  Saves valuable time for project managers by automating documentation creation, allowing them to focus on **strategy** and **execution**.

- **Groq LLM Integration**:  
  Uses **Groq LLM** to ensure accurate, contextually relevant outputs tailored to the specific needs of each project, guaranteeing high-quality results.


MiniPM streamlines the project planning process by automating documentation creation, ensuring well-organized, clear, and actionable project goals. This leads to **higher productivity** and **successful project execution**.


---

# Thank You for visiting The Hackers Playbook! 🌐

If you liked this research material;

- [Subscribe to our newsletter.](https://thehackersplaybook.substack.com)

- [Follow us on LinkedIn.](https://www.linkedin.com/company/the-hackers-playbook/)

- [Leave a star on our GitHub.](https://www.github.com/thehackersplaybook)

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>